Name: Kyle Salgado-Gouker <br>
Date: October 21, 2023 <br>
Class: DSC540 - Professor Williams <br>
Project: Milestone 3

### Cleaning/Formatting Website Data

Perform at least 5 data transformation and/or cleansing steps to your website data. The below examples are not required - they are just potential transformations you could do. If your data doesn't work for these scenarios, complete different transformations. You can do the same transformation multiple times if needed to clean your data. The goal is a clean dataset at the end of the milestone.

* Replace Headers
* Format data into a more readable format
* Identify outliers and bad data
* Find duplicates
* Fix casing or inconsistent values
* Conduct Fuzzy Matching

Make sure you clearly label each transformation step (Step #1, Step #2, etc.) in your code and describe what it is doing in 1-2 sentences. You can submit a Jupyter Notebook or a PDF of your code. If you submit a .py file you need to also include a PDF or attachment of your results.

Milestone 3 is due Sunday, by Midnight of Week 8. Refer to the rubric for more grading detail.

In [1]:
import time

# file system searches etc
import os
from os.path import basename, exists
import glob

# regular expressions
import re

# data frames
import pandas as pd
# smart arrays etc (will be required)
import numpy as np

# web access and html parsing (urllib, its submodules)
import requests
import urllib
import urllib.request
import urllib.error
from urllib.request import urlretrieve
# for a workaround.
import ssl

# work around read_html deprecation issue
from io import StringIO

# parser of web pages
from bs4 import BeautifulSoup
# more efficient parsing.
import lxml

# Fuzzy string matching
# If necessary, here are the installation commands.
# !pip install SciPy
# !pip install python-Levenshtein
# !pip install fuzzywuzzy
from fuzzywuzzy import fuzz

# for accessing sql (will be required)
import sqlite3

# fancy table printing
from tabulate import tabulate

In [2]:
# For testing: Make warnings fatal.

import warnings
warnings.filterwarnings("error")

In [3]:
# Store final project data in its own directory.

FINAL_DATA_DIRECTORY = "data/final"
QT_DATA_DIRECTORY = "data/final/QT"

# Check if the directory exists
if not os.path.exists(FINAL_DATA_DIRECTORY):
    # If it doesn't exist, make it
    os.makedirs(FINAL_DATA_DIRECTORY)
    print(f"Directory '{FINAL_DATA_DIRECTORY}' created.")


#### These are the URLs and Local Files for Queue-Times and Wikipedia Parks pages.

In [4]:
# constants for accessing files and the web.
QUEUE_TIMES_API = "https://queue-times.com/en-US/pages/api"
WIKIPEDIA_PARK_RANKINGS = "https://en.wikipedia.org/wiki/List_of_amusement_park_rankings"
WIKIPEDIA_PARK_RANKINGS_FILE = FINAL_DATA_DIRECTORY+"/amusement_park_rankings.html"
QUEUE_TIMES_PARK_LIST_URL = "https://queue-times.com/en-US/parks?group=country"

#### I set the flag below to false to skip the web scraping downloads.

wip_parks.csv is the final list of parks before removing records, which is calculated through webscraping Queue Times and Wikipedia.

In [5]:
DO_QT_DOWNLOAD = False
PARKS_WORK_IN_PROGRESS_FILE = FINAL_DATA_DIRECTORY+'/wip_parks.csv'
PARKS_WORK_IN_PROGRESS_URL = "https://drive.google.com/uc?id=1xoZG6VqbdMK0AgKb-eDnpvo_xenbcMiX&export=download"

In [6]:
# web download function

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36"
}

# download: A good citizen download function
#     url - the url accessed
#     destination - local file to write
#
# respects code 429 and waits instead of pounding.

# Function to disable SSL certificate verification
def disable_ssl_verification():
    ssl._create_default_https_context = ssl._create_unverified_context

# Call the function to disable SSL verification
# This is to workaround an SSL certificate error I am getting.
disable_ssl_verification()

def download(url, destination, secure=True):
    try:
        # Send a GET request with headers
        response = requests.get(url, headers=headers, verify=secure)
        # Check if the request was successful
        if response.status_code == 200:
            with open(destination, 'w') as f:
                f.write(response.text)
            print("Downloaded " + destination)
        elif response.status_code == 429:
            # Extract the Retry-After header value
            # This is to avoid hammering sites.
            retry_after = response.headers.get("Retry-After")
            if retry_after:
                # Convert the Retry-After value to seconds
                retry_after_seconds = int(retry_after)
                print("Rate limit exceeded. Waiting for " + str(retry_after_seconds) + " seconds.")
                time.sleep(retry_after_seconds)
                # Retry the request after waiting
                download(url, destination)
            else:
                print("Rate limit exceeded. Retry-After header not found.")
        else:
            print("Website returned " + str(response.status_code))
    except urllib.error.HTTPError:
        print("Failed to download " + url)
    except Exception:
        print("Error writing " + destination)
    return

def downloadFile(url, filename):
    if not exists(filename):
        local, _ = urlretrieve(url, filename)
        print("Downloaded " + local + "\n")
        return local, _

def downloadRawFile(url, filename):
    if not os.path.exists(filename):
        # Modify the URL to the raw content URL (replace "github.com" with "raw.githubusercontent.com")
        raw_url = url + "?raw=true"
        # Download the raw content
        local, _ = downloadFile(raw_url, filename)


### Download the data files for this project (so far)

Download them locally. Two reasons:
* May not be accessible this weekend.
* Second file sometimes changes. (no surprises!)

In [7]:
# Get wikipedia ranking page.
downloadRawFile(WIKIPEDIA_PARK_RANKINGS, WIKIPEDIA_PARK_RANKINGS_FILE)
downloadFile(PARKS_WORK_IN_PROGRESS_URL, PARKS_WORK_IN_PROGRESS_FILE)

# Workaround weird character conversion issue that started recently.
encoding = "utf-8"
# Read the HTML from the URL. Force "utf-8" encoding to workaround issue.
with open(WIKIPEDIA_PARK_RANKINGS_FILE, 'r', encoding = encoding) as rankings_file:
    # Read the contents of the file into a buffer
    rankings_html = rankings_file.read()
    

Downloaded data/final/wip_parks.csv



### Process Wikipedia Rankings.

The most important data here are the park names and the year-by-year attendance figures.

In [8]:
# Parse wikipedia rankings using Beautiful Soup with the lxml parser
soup = BeautifulSoup(rankings_html, 'lxml')

# Get the tables
tables = soup.find_all('table')

# Store DataFrames for each table in a list.
dataframes = []

# Skip table 0 - Corporations
# Skip table 1 - Worldwide
# Skip tables 6+ - Waterparks (for now)

wiki_columns = ['Rank', 'Amusement park', 'Location', '2009', '2010', '2011', '2012', 
                '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']

# Use these 4 tables (most popular parks in North America, Latin America, Asia, and Europe/Middle East)
for table in tables[2:5]:
    # Make a data frame from each table.
    html_str = str(table)
    # Workaround deprecation issuee.
    html_io = StringIO(html_str)
    # Read the HTML from the StringIO object
    df = pd.read_html(html_io)[0]

    df.columns = wiki_columns
    # Each html page scraped is a different beast.
    # This solves the issue with the wikipedia page parsing.
    # Sometimes I need the text of the td item, but sometimes the name of the park is iin a title attribute
    if 'Amusement Park' in df.columns:
        df['Amusement Park'] = df['Amusement Park'].apply(lambda x: x['title'] if isinstance(x, dict) and 'title' in x else x)
    # Add dataframe to a list.
    dataframes.append(df)
    
# Concatenate the DataFrames into one DataFrame.
all_wiki_amusement_parks_df = pd.concat(dataframes, ignore_index=True)

# Drop the 'Rank' column. Do this before finding duplicates!
all_wiki_amusement_parks_df.drop(columns=['Rank'], inplace=True)

# Remove duplicate rows based on all columns
all_wiki_amusement_parks_df.drop_duplicates(inplace=True)

# Convert all columns except 'Amusement Park' and 'Location' to numbers. Needed for stats and plotting later.
columns_to_convert = all_wiki_amusement_parks_df.columns.difference(['Amusement park', 'Location'])
all_wiki_amusement_parks_df[columns_to_convert] = all_wiki_amusement_parks_df[columns_to_convert].apply(pd.to_numeric, errors='coerce')

# Replace NaN values with 0 in the combined DataFrame. Attendance = 0 when park is closed for pandemic, never opened, or gone for good.
# List of columns to fill with 0
columns_to_fillna_with_0 = ['2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']

# Loop through each column and fill NaN with 0
for column in columns_to_fillna_with_0:
    all_wiki_amusement_parks_df[column].fillna(0, inplace=True)

# Sort the data frame by Amusement park.
all_wiki_amusement_parks_df = all_wiki_amusement_parks_df.sort_values(by='Amusement park')

# Fix the column names.
wiki_columns = ['Amusement park', 'Location', '2009', '2010', '2011', '2012', 
                '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']

all_wiki_amusement_parks_df.columns = wiki_columns

### Let's see the full list of amusement parks!

In [9]:
# print a few columns from th tables loaded.
wiki_desired_columns = ['Amusement park', 'Location', '2009', '2021']
print(tabulate(all_wiki_amusement_parks_df[wiki_desired_columns], headers='keys', tablefmt='pretty', showindex=False))


+------------------------------------------------------------+-------------------------------------------+------------+------------+
|                       Amusement park                       |                 Location                  |    2009    |    2021    |
+------------------------------------------------------------+-------------------------------------------+------------+------------+
|            Alton Towers at Alton Towers Resort             |           Alton, United Kingdom           | 2650000.0  | 1800000.0  |
|                     Beto Carrero World                     |          Santa Catarina, Brazil           | 1000000.0  | 1895000.0  |
|                  Busch Gardens Tampa Bay                   |       Tampa, Florida, United States       | 4100000.0  | 3210000.0  |
|                    Canada's Wonderland                     |         Vaughan, Ontario, Canada          | 3160000.0  |  587000.0  |
|                        Cedar Point                         |       

#### Observations:

* It looks like the attndance values are floating point. That's not correct.
* The park names are not the same as the roller coaster data base, nor the queue times data base.
* The location is more informative than the "country" field in queue times.

Last milestone I solved the issue with inconsistent park names between queue times and the roller coaster data base.
I use the same technique here.

### Introduce 2 Fields for Park Matching.

* Park number: Matching Park number in Queue Times data, which will be the unique Park identifier.
* Best match: For Fuzzy string matching.

In [10]:
all_wiki_amusement_parks_df['Park number'] = ""
all_wiki_amusement_parks_df['Best match'] = ""


#### Web Scrape Queue Times for Park Numbers.

In [11]:
# URL to web scrape.
url = QUEUE_TIMES_PARK_LIST_URL

# Read it.
response = requests.get(url)

# If the request was successful
if response.status_code == 200:

    soup = BeautifulSoup(response.text, 'html.parser')

    # Initialize lists to store the data
    amusement_parks = []
    park_numbers = []
    countries = []

    # Each web page is a little different.
    # The QT park page by country has a separate panel for each country.
    # Inside each country panel is the park a_tag which has the data (see below)
    #
    # 1. Find all the <div> panels (countries)
    div_panels = soup.find_all('div', class_='panel')

    # Iterate through the countries
    for panel in div_panels:
        # Find the <h2> tag inside the panel
        country_name = panel.find('h2').text.strip()
        
        # Find all the <a> Park tags inside the panel
        a_tags = panel.find_all('a', class_='panel-block')
        
        # Iterate through the Park <a> tags
        for a_tag in a_tags:
            # Extract the Amusement park and link
            amusement_park = a_tag.text.strip()
            link = a_tag['href'] 
            amusement_park = amusement_park.split("\n")[0]
            # Append the data to the lists
            amusement_parks.append(amusement_park)
            # The park number is part of the link.
            park_numbers.append(link.split("/")[-1])
            countries.append(country_name)

    # Make the dataframe from the lists collected in the scrapiing.
    QT_park_list_df = pd.DataFrame({
        'Amusement park': amusement_parks,
        'Park number': park_numbers,
        'Country': countries
    })

    # ALL DONE!
    
    # Display the DataFrame
    print(tabulate(QT_park_list_df, headers='keys', tablefmt='pretty', showindex=False))
else:
    print("Failed to fetch the URL from Queue Times.")

+-------------------------------------------+-------------+---------------------+
|              Amusement park               | Park number |       Country       |
+-------------------------------------------+-------------+---------------------+
|                Familypark                 |     322     |       Austria       |
|                Bellewaerde                |     276     |       Belgium       |
|               Bobbejaanland               |     311     |       Belgium       |
|            Plopsaland De Panne            |     54      |       Belgium       |
|              Walibi Belgium               |     14      |       Belgium       |
|            Beto Carrero World             |     319     |       Brazil        |
|            Canada's Wonderland            |     58      |       Canada        |
|            La Ronde, Montreal             |     48      |       Canada        |
|          Shanghai Disney Resort           |     30      |        China        |
|             Dj

### Match Amusement Parks by name and move Park Number field from QT DF into Wiki DF.

* Exact match
* Near match (QT inside Wiki)
* Near match (Wiki inside QT)
* Fuzzy match.


#### Note: The fuzzy matching algorithm cannot resolve a few important differences between some wikipedia/queuetime entries.

They will be then be skipped in the Park Matching algorithm.


In [12]:
# These parks need to be pre-set.

all_wiki_amusement_parks_df.loc[all_wiki_amusement_parks_df['Amusement park'] == 'Disneyland Park', 'Park number'] = '16'
all_wiki_amusement_parks_df.loc[all_wiki_amusement_parks_df['Amusement park'] == 'Disneyland Hong Kong', 'Park number'] = '31'
all_wiki_amusement_parks_df.loc[all_wiki_amusement_parks_df['Amusement park'] == 'Disneyland Park at Disneyland Paris', 'Park number'] = '4'
all_wiki_amusement_parks_df.loc[all_wiki_amusement_parks_df['Amusement park'] == 'Magic Kingdom Theme Park at Walt Disney World Resort', 'Park number'] = '6'
all_wiki_amusement_parks_df.loc[all_wiki_amusement_parks_df['Amusement park'] == 'Walt Disney Studios Park at Disneyland Paris', 'Park number'] = '28'


<span style="color:red">
    <b><u>Transformation 1</u></b>: Remove parks stored in Wikipedia without Queue-Time data.
</span>

Records in wikipedia alone do not have enough info to glean impact of attraction.

In [95]:

# # These parks need to be removed because Queue Times doesn't include them.
indices_to_remove = [index for index, row in all_wiki_amusement_parks_df.iterrows() if row['Amusement park'] in 
                     ['Fantasilandia', 'La Feria', 'Parque Mundo Aventura', 'Parque Plaza Sésamo', 'Parque Warner', 
                      'Parque Xcaret', 'Parque de la Costa', 'Theme Parque Nacional del Café', 'Puy du Fou',
                      'Mundo Petapa', 'Futuroscope', 'La Feria Chapultepec Mágico', 'Tivoli Gardens']]

# Remove rows with the specified indices
all_wiki_amusement_parks_df.drop(indices_to_remove, inplace=True)



<span style="color:red">
    <b><u>Transformation 2</u></b>: Introduce 'Park number', a universal key across the data sets.
</span>

### Park name matching algorithm.

Sets Park number field, the QT number for the corresponding park (often with a different name.)

In [96]:
for w, row_wiki in all_wiki_amusement_parks_df.iterrows():
    amusement_park_w = row_wiki['Amusement park']
    
    # Initialize variables to store the best match and its index
    best_match_index = None
    best_match_score = -1
    
    # Iterate through each row in QT_park_list_df
    for q, row_QT in QT_park_list_df.iterrows():
        amusement_park_Q = row_QT['Amusement park']
        
        # exact match first
        if row_wiki['Park number'] == "" and amusement_park_w == amusement_park_Q:
            # print(f"Exact match for {amusement_park_w} and {amusement_park_Q}")
            all_wiki_amusement_parks_df.at[w, 'Park number'] = row_QT['Park number']
            break
            
        # if queue time string is in wiki string
        elif row_wiki['Park number'] == "" and amusement_park_w.find(amusement_park_Q) != -1:
            # print(f"Near match (wiki contains QT) for {amusement_park_w} and {amusement_park_Q}")
            all_wiki_amusement_parks_df.at[w, 'Park number'] = row_QT['Park number']
            break
            
        # if wiki string is in queue time string
        elif row_wiki['Park number'] == "" and amusement_park_Q.find(amusement_park_w) != -1:
            # print(f"Near match (QT contains wiki) for {amusement_park_w} and {amusement_park_Q}")
            all_wiki_amusement_parks_df.at[w, 'Park number'] = row_QT['Park number']
            break
            
        # fuzzy match
        elif row_wiki['Park number'] == "":
            fuzz_score = fuzz.ratio(amusement_park_w, amusement_park_Q)
            # print(f"Fuzzy match for {amusement_park_w} and {amusement_park_Q}: {fuzz_score}")
            
            if fuzz_score > best_match_score:
                best_match_score = fuzz_score
                best_match_index = q
                
    if all_wiki_amusement_parks_df.at[w, 'Park number'] == "" and best_match_index is not None:
        all_wiki_amusement_parks_df.at[w, 'Park number'] = QT_park_list_df.at[best_match_index, 'Park number']


#### Show Results of Name Matching

In [97]:
selected_columns = ['Park number', 'Amusement park']

print(tabulate(all_wiki_amusement_parks_df[selected_columns], headers=selected_columns, tablefmt='pretty', showindex=False))

+-------------+------------------------------------------------------------+
| Park number |                       Amusement park                       |
+-------------+------------------------------------------------------------+
|      1      |            Alton Towers at Alton Towers Resort             |
|     319     |                     Beto Carrero World                     |
|     24      |                  Busch Gardens Tampa Bay                   |
|     58      |                    Canada's Wonderland                     |
|     50      |                        Cedar Point                         |
|      3      |              Chessington World of Adventures               |
|     160     |                        De Efteling                         |
|     17      |              Disney California Adventure Park              |
|      8      |    Disney's Animal Kingdom at Walt Disney World Resort     |
|      7      |   Disney's Hollywood Studios at Walt Disney World Resort   |

### Now we have a uniform method of referencing parks.

<span style="color:red">
    <b><u>Transformation 3</u></b>: Change Park (from wikipedia) column names for consistency across sets.
</span>

In [98]:
all_wiki_amusement_parks_df = all_wiki_amusement_parks_df.rename(columns={'Amusement park': 'Park_Name',
                                            'Park number': 'Park_Number'})

if 'Best match' in all_wiki_amusement_parks_df.columns:
    all_wiki_amusement_parks_df.drop(columns=['Best match'], inplace=True)


<span style="color:red">
    <b><u>Transformation 4</u></b>: Add attendance fields from 2006-2008 and 2022+ using Queue Times
</span>

Note: This has the same source as the Wikipedia page. The Wikipedia table does not include this data.

* TEA/ERA. 2008 Attraction Attendance Report. Themed Entertainment Association (TEA); 2009 https://www.teaconnect.org/images/files/TEA_23_503031_140617.pdf. Accessed 30 December, 2021.
* TEA/ERA. 2007 Attraction Attendance Report. Themed Entertainment Association (TEA); 2008 https://www.teaconnect.org/images/files/TEA_29_601512_140617.pdf. Accessed 30 December, 2021.
* TEA/ERA. 2006 Theme Park Attendance Report. Themed Entertainment Association (TEA); 2007 https://www.teaconnect.org/images/files/TEA_158_724007_160525.pdf. Accessed 30 December, 2021.
* TEA/AECOM. 2022 Theme Index and Museum Index: The Global Attractions Attendance Report. Themed Entertainment Association (TEA); 2023 https://aecom.com/wp-content/uploads/documents/reports/AECOM-Theme-Index-2022.pdf. Accessed 8 July, 2023.

In [99]:
# List of columns for missing years I will add from Queue Times.
columns_to_add = ['2006', '2007', '2008', '2022']

# Initialize these new columns with NaN values
for column in columns_to_add:
    all_wiki_amusement_parks_df = all_wiki_amusement_parks_df.assign(**{column: np.nan})

#### Web Scrape of Queue Times.

* Use Queue Times API to obtain Parks and Attendance. (For Transformation 4)
* Webscrape the Queue Times Pages for Ride Entries in each park. (TODO)
* Add missing rides to rides data base. (TODO)
* Add missing parks to parks data base. (For Transformation 5)
* Build transaction file of wait times using web scraping. (TODO)

Webscraping - First grab all the necessary web documents from Queue_Times.

Use friendly download to avoid battering the server.

Start with QT_park_list_df, a simple data base read from the QT parks main page.

     Amusement park, Park number, Country   

Files are stored in QT_DATA_DIRECTORY:
    Optional attendance File has format: "attendance_"+park_number+".html"
    Overall stats file has format: "overall_stats_"+park_number+".html"
    Yearly stats files have this format: park_number+"_"+year+"_stats.html"

For each park_number in list:
* Download queue-times.com/parks/{park_number}/attendances
* Download queue-times.com/parks/{park_number}/stats
* stats_html = read_html
* use soup to find all tags with format <a href="/parks/{park_number}/stats/* 
* these indicate the years of statistics on the site
* for each matching year:
* Download queue-times.com/parks/{park_number}/stats/{year}


In [68]:
if DO_QT_DOWNLOAD:

    # Get all park numbers from queue time list.
    qt_parks = QT_park_list_df['Park number']

    # For each park.
    for qt_park in qt_parks:
        # park *may* have attendance data. If so, grab it.
        url_attendance = "https://queue-times.com/parks/" + str(qt_park) + "/attendances"
        attendance_filename = QT_DATA_DIRECTORY + "/attendance_" + str(qt_park) + ".html"

        # Download attendance data using requests library.
        response_attendance = requests.get(url_attendance)
        with open(attendance_filename, 'wb') as attendance_file:
            attendance_file.write(response_attendance.content)

        # All parks should have statistics.
        url_statistics = "https://queue-times.com/parks/" + str(qt_park) + "/stats"
        statistics_filename = QT_DATA_DIRECTORY + "/overall_stats_" + str(qt_park) + ".html"

        # Download statistics data using requests library.
        response_statistics = requests.get(url_statistics)
        with open(statistics_filename, 'wb') as statistics_file:
            statistics_file.write(response_statistics.content)

        try:
            # Read the HTML from the URL.
            with open(statistics_filename, 'r') as overall_stats_file:
                # Read the contents of the file into a buffer
                overall_stats_html = overall_stats_file.read()
                soup = BeautifulSoup(overall_stats_html, 'html.parser')

                # Find all tags with format <a href="/parks/{park_number}/stats/*
                for tag in soup.find_all('a', href=True):
                    if f"/parks/{qt_park}/stats/" in tag['href']:
                        year = tag['href'].split("/stats/")[1]
                        year_url = f'https://queue-times.com{tag["href"]}'
                        year_stats_filename = QT_DATA_DIRECTORY + f"/{str(qt_park)}_{year}_stats.html"

                        # Download year-specific statistics data.
                        response_year_stats = requests.get(year_url)
                        with open(year_stats_filename, 'wb') as year_stats_file:
                            year_stats_file.write(response_year_stats.content)
        except Exception as e:
            print(f"Error reading {statistics_filename}: {str(e)}")

In [69]:
if DO_QT_DOWNLOAD:

    # I should have downloaded the files into subfolders, but I'll move them around now.
    # First build the directory structure.
    # Create attendance, overall, and stats directories if they don't exist
    for subdir in ['attendance', 'overall', 'stats']:
        dir_path = os.path.join(QT_DATA_DIRECTORY, subdir)
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)
            print(f"Directory '{dir_path}' created.")

    # Define the year range
    year_range = range(2014, 2024)  # From 2014-2023.

    # Create subdirectories for each year within the 'stats' directory
    for year in year_range:
        # make subfolder name for each year.
        dir_name = os.path.join(QT_DATA_DIRECTORY, 'stats', str(year))
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)
            print(f"Directory '{dir_name}' created.")

In [100]:
if DO_QT_DOWNLOAD:

    # Now move the downloaded files.
    # ParkNumber_YearNumber_stats.html -> data/final/QT/stats/YearNumber/ParkNumber.html
    # attendance_ParkNumber.html -> data/final/QT/attendance/ParkNumber.html
    # overall_stats_ParkNumber.html -> data/final/QT/overall/ParkNumber.html

    # Get all park numbers from the queue time list.
    qt_parks = QT_park_list_df['Park number']

    # For each park.
    for qt_park in qt_parks:
        # Move year stats files into each year subfolder of stats.
        for year in year_range:
            test_filename = os.path.join(QT_DATA_DIRECTORY, f"{qt_park}_{year}_stats.html")
            if os.path.isfile(test_filename):
                destination = os.path.join(QT_DATA_DIRECTORY, "stats", str(year), f"{qt_park}.html")
                os.rename(test_filename, destination)
        # Move attendance files into attendance subfolder
        attendance_filename = os.path.join(QT_DATA_DIRECTORY, f"attendance_{qt_park}.html")
        if os.path.isfile(attendance_filename):
            destination = os.path.join(QT_DATA_DIRECTORY, "attendance", f"{qt_park}.html")
            os.rename(attendance_filename, destination)
        # move overall stats files into overall subfolder
        overall_stats_filename = os.path.join(QT_DATA_DIRECTORY, f"overall_stats_{qt_park}.html")
        if os.path.isfile(overall_stats_filename):
            destination = os.path.join(QT_DATA_DIRECTORY, "overall", f"{qt_park}.html")
            os.rename(overall_stats_filename, destination)


In [101]:
# copy the data frames before we break them (FOR DEBUGGING)

# Assuming df is your DataFrame
df_wiki_copy = all_wiki_amusement_parks_df.copy()

In [102]:
all_wiki_amusement_parks_df = df_wiki_copy.copy() # (FOR DEBUGGING. TO AVOID RELOADING)

In [103]:
# Define the list of columns to convert to integers
string_columns = ['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', 
                   '2017', '2018', '2019', '2020', '2021', '2022']

# Iterate through numeric columns and apply conversion
for column in string_columns:
    all_wiki_amusement_parks_df[column] = all_wiki_amusement_parks_df[column].astype('str')


In [104]:
selected_columns = all_wiki_amusement_parks_df.columns
print(tabulate(all_wiki_amusement_parks_df[selected_columns].head(5), headers=selected_columns, tablefmt='pretty', showindex=False))

print("QT Before Rename")
selected_columns = QT_park_list_df.columns
print(tabulate(QT_park_list_df[selected_columns].head(5), headers=selected_columns, tablefmt='pretty', showindex=False))


# Define a dictionary to map the old column names to the new names
column_mapping = {
    'Amusement park': 'Park_Name',
    'Park number': 'Park_Number',
    'Country': 'Location'
}

# Use the rename method to rename the columns
QT_park_list_df = QT_park_list_df.rename(columns=column_mapping)

# Now the columns should be renamed
print("QT after Rename")
selected_columns = QT_park_list_df.columns
print(tabulate(QT_park_list_df[selected_columns].head(5), headers=selected_columns, tablefmt='pretty', showindex=False))

# Define the desired column order
desired_order = ["Park_Number", "Park_Name", "Location"] + [str(year) for year in range(2006, 2023)]

# Reorder the columns
all_wiki_amusement_parks_df = all_wiki_amusement_parks_df[desired_order]

all_wiki_amusement_parks_df.to_csv(FINAL_DATA_DIRECTORY+'/original_web_parks.csv', index=False)


+-------------------------------------+-------------------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+---------+-----------+-----------+-------------+------+------+------+------+
|              Park_Name              |           Location            |   2009    |   2010    |   2011    |   2012    |   2013    |   2014    |   2015    |   2016    |   2017    |   2018    |  2019   |   2020    |   2021    | Park_Number | 2006 | 2007 | 2008 | 2022 |
+-------------------------------------+-------------------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+---------+-----------+-----------+-------------+------+------+------+------+
| Alton Towers at Alton Towers Resort |     Alton, United Kingdom     | 2650000.0 | 2750000.0 | 2650000.0 | 2400000.0 | 2500000.0 | 2575000.0 | 1925000.0 | 1980000.0 | 2000000.0 | 2100000.0 | 2130

#### Load Attendance Data (if available) for all parks.

<span style="color:red">
    <b><u>Transformation 5</u></b>: Side effect of Transformation 4 - adds records that are candidates for data set.
</span>

This code loads the webscraped html tables and transfers the Queue-Timess 2006-2008 and 2022 data. 

In [105]:
def convert_data_types(df):
    # Convert all columns (except 'Park_Number') to string
    columns_to_string = df.columns.difference(['Park_Number'])
    df[columns_to_string] = df[columns_to_string].astype(str)
    # Convert 'Park_Number' to int
    df['Park_Number'] = df['Park_Number'].astype(int)
    return df

all_wiki_amusement_parks_df = convert_data_types(all_wiki_amusement_parks_df)
wiki_database_columns = ['Park_Number', 'Park_Name', 'Location', '2006', '2007', '2008', '2009', '2010', 
                        '2011', '2012', '2013', '2014', '2015', 
                        '2016', '2017', '2018', '2019', '2020', '2021', '2022']

if DO_QT_DOWNLOAD:
    parks_df = pd.DataFrame(columns=wiki_database_columns)

    print(len(all_wiki_amusement_parks_df), " park records before processing attendance.")

    attendance_dir = os.path.join(QT_DATA_DIRECTORY, "attendance")

    for filename in os.listdir(attendance_dir):
        if filename.endswith(".html"):
            # Extract park_number from the filename
            park_number = filename.split(".")[0]

            qt_matching_record = QT_park_list_df[QT_park_list_df['Park_Number'] == park_number].iloc[0]

            print("Processing park ", park_number, ". Park name = ", qt_matching_record['Park_Name'])

            # Check if the record exists in the DataFrame

            matching_record_df = all_wiki_amusement_parks_df[all_wiki_amusement_parks_df['Park_Number'] == int(park_number)]

            if matching_record_df.empty:
                new_data = {'Park_Number': int(park_number),
                            'Park_Name': qt_matching_record['Park_Name'],
                            'Location': qt_matching_record['Location']}
                # set attendance data to nan if unknown.
                for year in range(2006, 2023):
                    new_data[str(year)] = np.nan

                print("Adding new record: Attendance Data for park ", park_number, ". Park name = ", qt_matching_record['Park_Name'])
                # initialize dataframe with new record.
                matching_record_df = pd.DataFrame([new_data])
                matching_record_df = convert_data_types(matching_record_df)

            # restrict to necessary coolumns
            matching_record_df = matching_record_df[wiki_database_columns]
            # get the 0th record of the frame (make it a series) - NOTE: There is only ever one.
            # Avoid slice by working on a copy.
            matching_record = matching_record_df.iloc[0].copy()

            # Read the downloaded file,
            with open(os.path.join(attendance_dir, filename), 'r', encoding='utf-8') as attendance_file:
                attendance_html = attendance_file.read()

            # Use soup to get the ttable.
            soup = BeautifulSoup(attendance_html, 'html.parser')
            attendance_table = soup.find('table', {'class': 'table is-fullwidth'})

            if attendance_table:
                # Use Pandas to read the HTML table into a DataFrame
                html_str = str(attendance_table)
                # Workaround deprecation issuee.
                html_io = StringIO(html_str)
                # Read the HTML from the StringIO object
                # Take the first DataFrame in the list:
                attendance_df = pd.read_html(html_io)[0]

                # Convert 'Year' and 'Attendance' columns to appropriate data types
                # The table entries have junk besides numbers in them.
                attendance_df['Year'] = attendance_df['Year'].astype(str).str.extract(r'(\d+)').astype(int)
                # The Attendance has a 'footnote' - I use a regular expression to grab the number.
                attendance_df['Attendance'] = attendance_df['Attendance'].astype(str).str.replace(',', '', regex=True).str.extract(r'(\d+)').astype(int)

                # use the park name and location from queue times as the 'correct' names
                # I already set park name in roller coaster data base to match the QT park name.
                matching_record['Park_Name'] = qt_matching_record['Park_Name']
                # Do not change Location. It is more descriptive than QT's, Country alone.

                # Set each year in matching_record to the Attendance in attendance_df
                for year in range(2006, 2023):
                    attendance_year_test_df = attendance_df[attendance_df['Year'] == year]
                    if not attendance_year_test_df.empty:
                        attendance_value = attendance_year_test_df['Attendance'].values[0]
                        matching_record[str(year)] = attendance_value
                        # print("Park: ", park_number, ". Name: ", qt_matching_record['Park_Name'],
                        #       ". Year: ", year, ". Attendance: ", attendance_value)

            # add new record
            parks_df = pd.concat([parks_df, pd.DataFrame([matching_record], columns=wiki_database_columns)], ignore_index=True)
            parks_df.to_csv(FINAL_DATA_DIRECTORY+'/wip_parks.csv', index=False)
else:
    parks_df = pd.read_csv(FINAL_DATA_DIRECTORY+'/wip_parks.csv')
    
print(len(parks_df), " park records after processing attendance.")


119  park records after processing attendance.


#### The new dataframe, park_df, has 81 new possible candidates for inclusion.


Note: Most of these will be removed later, because of too many nan columns.

#### The code below shows the current state. Only selected columns are shown.

In [106]:
selected_columns=['Park_Number', 'Park_Name', '2006', '2007', '2008', '2015', '2020', '2022']
print(tabulate(parks_df[selected_columns].head(25), headers=selected_columns, tablefmt='pretty', showindex=False))

+-------------+------------------------------+----------+----------+----------+----------+---------+----------+
| Park_Number |          Park_Name           |   2006   |   2007   |   2008   |   2015   |  2020   |   2022   |
+-------------+------------------------------+----------+----------+----------+----------+---------+----------+
|     317     |         Energylandia         |    0     |    0     |    0     |    0     |    0    |    0     |
|     23      |  Busch Gardens Williamsburg  |    0     | 3157000  | 3094000  |    0     |    0    |    0     |
|     301     |      Walibi Rhône-Alpes      |    0     |    0     |    0     |    0     |    0    |    0     |
|     35      |    Six Flags Over Georgia    |    0     |    0     |    0     |    0     |    0    |    0     |
|     62      |        Kings Dominion        |    0     |    0     |    0     |    0     |    0    |    0     |
|      9      |         Parc Astérix         | 1800000  | 1620000  | 1800000  | 1850000  | 1163000 | 263

<span style="color:red">
    <b><u>Transformation 6</u></b>: Change Park (from wikipedia) data types for attendance numbers etc.
</span>

* Park_Number should be an int64
* Park_Name should be a string.
* Location should be a string.
* 2006-2022 should be int64 (transform from float as necessary)

Coerce and FillNA as needed. Change NAN to zeroes.

First show the current state:

#### Function to Show Column Name and Data Type for a DataFrame.

In [107]:
def show_metadata(df):
    metadata_df = pd.DataFrame({
        "Data Type": df.dtypes
    })
    print(tabulate(metadata_df, headers=['Field Name', 'Data Type'], tablefmt='pretty'))


#### Some of the Fields are Incorrectly Typed in Both DataFrames.

In [108]:
print("\nFields of Park Data Frame")
show_metadata(parks_df)


Fields of Park Data Frame
+-------------+-----------+
| Field Name  | Data Type |
+-------------+-----------+
| Park_Number |   int64   |
|  Park_Name  |  object   |
|  Location   |  object   |
|    2006     |   int64   |
|    2007     |   int64   |
|    2008     |   int64   |
|    2009     |   int64   |
|    2010     |   int64   |
|    2011     |   int64   |
|    2012     |   int64   |
|    2013     |   int64   |
|    2014     |   int64   |
|    2015     |   int64   |
|    2016     |   int64   |
|    2017     |   int64   |
|    2018     |   int64   |
|    2019     |   int64   |
|    2020     |   int64   |
|    2021     |   int64   |
|    2022     |   int64   |
+-------------+-----------+


In [109]:

# Define the data type conversions for the specified columns
data_type_conversions = {
    'Park_Name': 'string',
    'Location': 'string',
    'Park_Number': 'int64'
}

# Define the list of columns to convert to integers
attendance_columns = ['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']

# Iterate through numeric columns and apply conversion
for column in attendance_columns:
    parks_df[column] = pd.to_numeric(parks_df[column], errors='coerce').fillna(0).astype('int64')

# Apply data type conversions to the DataFrame
parks_df = parks_df.astype(data_type_conversions)


#### Show New Types

In [110]:
print("\nFields of Park Data Frame")
show_metadata(parks_df)


Fields of Park Data Frame
+-------------+-----------+
| Field Name  | Data Type |
+-------------+-----------+
| Park_Number |   int64   |
|  Park_Name  |  string   |
|  Location   |  string   |
|    2006     |   int64   |
|    2007     |   int64   |
|    2008     |   int64   |
|    2009     |   int64   |
|    2010     |   int64   |
|    2011     |   int64   |
|    2012     |   int64   |
|    2013     |   int64   |
|    2014     |   int64   |
|    2015     |   int64   |
|    2016     |   int64   |
|    2017     |   int64   |
|    2018     |   int64   |
|    2019     |   int64   |
|    2020     |   int64   |
|    2021     |   int64   |
|    2022     |   int64   |
+-------------+-----------+


That's good.

##### Note: TODO.

The Ride Data Frame will need further type changes:

* Opening_Date will be a TimeStamp
* Height, Length, Speed, Drop, Max_Vertical_Angle, G-force, and Height_Restriction will be float64
* Cost will be a currency
* Inversions and Capacity will be int64
* Duration will be a time.

Unfortunately, these will require more parsing.
Fortunately, they are not <i>yet</i> required for this project.


<span style="color:red">
    <b><u>Transformation 7</u></b>: Drop 2020 and 2021 columns. COVID impact.<p>    
</span>

Apply this only to a copy of the data.

In [111]:
parks_attendance_subset_df = parks_df.copy()

columns_to_drop = ['2020', '2021']

parks_attendance_subset_df = parks_attendance_subset_df.drop(columns=columns_to_drop)

<span style="color:red">
    <b><u>Transformation 8</u></b>: Drop rows with only zero or nan attendance data.<p>    
</span>

In [112]:
# Columns to check for NaN or zero values
# I have already removed 2020 and 2021
attendance_columns = [str(year) for year in range(2006, 2020)] + ['2022']

# Define boolean mask for NaN or zero values in the specified columns.
mask = (parks_df[attendance_columns] == 0) | parks_df[attendance_columns].isna()
# ALL attendance columns must be NaN or zero to drop the record.
mask = mask.all(axis=1)

# Drop rows that satisfy the condition
filtered_parks_df = parks_df[~mask]

print(len(filtered_parks_df), " park records after final drops.")

54  park records after final drops.


Store this result to csv.

#### Store Parks Data Frame to CSV file (backup)

In [113]:
filtered_parks_df.to_csv(FINAL_DATA_DIRECTORY+'/final_web_parks.csv', index=False)

#### ROW Tally Explanation.

So, the Wikipedia set had 38 records. That was augmented via QueueTime to 119.
The only records with attendance data are these 54.

This is the data that will be stored in SQl table, parks_subset.

#### TODO: Create Transaction Files from QUEUE-TIME.

* Shows how queue-lines change
* Shows effects of seasons and days of week.

##### These will also be SQL tables.

Here is the creation of SQL tables for the database and filtered database.

#### Set up an SQL Data Base to Facilitate Queries.

In [114]:
# Connect to SQLite database
conn = sqlite3.connect(FINAL_DATA_DIRECTORY+'/park_info.db')

# First get rid of previous data tables. We regen every time for now.
cursor = conn.cursor()

# Drop the table
cursor.execute('DROP TABLE IF EXISTS parks')
cursor.execute('DROP TABLE IF EXISTS parks_subset')

# Write dataframes to SQLite
parks_df.to_sql(name='parks', con=conn, if_exists='replace', index = False, index_label=['Park_Name', 'Park_Number'])
filtered_parks_df.to_sql(name='parks_subset', con=conn, if_exists='replace', index = False, index_label=['Park_Name', 'Park_Number'])

conn.commit()

#### Helper Functions for Implementing SQL Test

In [115]:
def printFormattedTestStat(value, dec=2):
    format_string = "{:.{dec}f}"
    return format_string.format(value, dec=dec)

def printCurrencyFormattedTestStat(value, dec=2):
    format_string = "{:,.{dec}f}"
    return format_string.format(value, dec=dec)


calculated_columns = []
currency_columns = []

def testSqlAccess(cursor, request, title, column_names=None, hdr = None):
    cursor.execute(request)
    test_data = cursor.fetchall()
    if column_names is None:
        column_names = [description[0] for description in cursor.description]
    test_df = pd.DataFrame(test_data, columns=[description[0] for description in cursor.description])
    selected_columns = [col for col in column_names if col in test_df.columns]
    if hdr != None:
        headers = hdr
    else:
        headers = selected_columns
    formatted_df = test_df[selected_columns].copy()
    for col in selected_columns:
        if col in calculated_columns:
            formatted_df[col] = formatted_df[col].map(lambda x: printFormattedTestStat(x))
        if col in currency_columns:
            formatted_df[col] = formatted_df[col].map(lambda x: printCurrencyFormattedTestStat(x))

    print("\n" + title + "\n")
    print(tabulate(formatted_df, headers=headers, tablefmt='psql', showindex=False))
    return formatted_df


In [116]:
# Test with SQL query.

# query all rides with Kings Island 


park_columns_1 = ["Park_Name", "Park_Number", "Location"]

testSqlAccess(cursor, "SELECT * FROM parks WHERE Location LIKE '%Florida%'", "Florida Parks", column_names = park_columns_1)



Florida Parks

+-------------------------------------------+---------------+----------------------------------+
| Park_Name                                 |   Park_Number | Location                         |
|-------------------------------------------+---------------+----------------------------------|
| Epcot                                     |             5 | Bay Lake, Florida, United States |
| Animal Kingdom                            |             8 | Bay Lake, Florida, United States |
| Islands Of Adventure At Universal Orlando |            64 | Orlando, Florida, United States  |
| Universal Studios At Universal Orlando    |            65 | Orlando, Florida, United States  |
| Busch Gardens Tampa                       |            24 | Tampa, Florida, United States    |
| Seaworld Orlando                          |            21 | Orlando, Florida, United States  |
| Disney Hollywood Studios                  |             7 | Bay Lake, Florida, United States |
| Disney Magic

,Park_Name,Park_Number,Location
0,Epcot,5,"Bay Lake, Florida, United States"
1,Animal Kingdom,8,"Bay Lake, Florida, United States"
2,Islands Of Adventure At Universal Orlando,64,"Orlando, Florida, United States"
3,Universal Studios At Universal Orlando,65,"Orlando, Florida, United States"
4,Busch Gardens Tampa,24,"Tampa, Florida, United States"
5,Seaworld Orlando,21,"Orlando, Florida, United States"
6,Disney Hollywood Studios,7,"Bay Lake, Florida, United States"
7,Disney Magic Kingdom,6,"Bay Lake, Florida, United States"
